# SAVE MODEL

============================================================


In [ ]:
# =========================
# train_fp_pro_final.py
# =========================

import pandas as pd
import numpy as np
import joblib
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import precision_recall_curve, roc_auc_score, average_precision_score
from imblearn.over_sampling import SMOTE

# =========================
# CONFIGURATION
# =========================
FP_RADIUS = 2
FP_NBITS = 2048
RANDOM_STATE = 42

# =========================
# HELPERS
# =========================
def mol_from_smiles(smiles):
    """Return RDKit Mol object or None"""
    try:
        return Chem.MolFromSmiles(smiles)
    except:
        return None

def canonical_smiles(smiles):
    """Return canonical SMILES or None"""
    m = mol_from_smiles(smiles)
    return Chem.MolToSmiles(m, canonical=True) if m else None

def mol_to_fp(mol):
    """Convert Mol to Morgan fingerprint (ExplicitBitVect)"""
    return AllChem.GetMorganFingerprintAsBitVect(mol, FP_RADIUS, nBits=FP_NBITS)

def fp_to_numpy(fp):
    """Convert RDKit BitVect to numpy array"""
    arr = np.zeros((FP_NBITS,), dtype=np.uint8)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

# =========================
# LOAD DATA
# =========================
df = pd.read_csv("Descriptors_Specified_SmallGTPases.csv")
print("Original dataset:", df.shape)
print(df['bioactivity_class'].value_counts())

# Fill missing canonical_smiles
df['canonical_smiles'] = df['canonical_smiles'].fillna(df.get('smiles', np.nan))
df['canonical_smiles'] = df['canonical_smiles'].apply(lambda s: canonical_smiles(s) if pd.notna(s) else None)
df = df.dropna(subset=['canonical_smiles'])
print("After canonicalization:", df.shape)

# =========================
# COMPUTE FINGERPRINTS
# =========================
fps = []
for smi in df['canonical_smiles']:
    m = mol_from_smiles(smi)
    fps.append(mol_to_fp(m) if m else None)
df['fp'] = fps
df = df.dropna(subset=['fp']).reset_index(drop=True)
print("After removing invalid SMILES:", df.shape)

# =========================
# PREPARE X and y
# =========================
X_fp = np.array([fp_to_numpy(x) for x in df['fp']])
y = df['bioactivity_class'].values

# =========================
# TRAIN-TEST SPLIT (STRATIFIED)
# =========================
X_train_fp, X_test_fp, y_train, y_test, idx_train, idx_test = train_test_split(
    X_fp, y, df.index, test_size=0.2, stratify=y, random_state=RANDOM_STATE
)
train_df = df.loc[idx_train].reset_index(drop=True)
test_df = df.loc[idx_test].reset_index(drop=True)

# =========================
# SMOTE TO BALANCE CLASSES
# =========================
smote = SMOTE(random_state=RANDOM_STATE)
X_train_res, y_train_res = smote.fit_resample(X_train_fp, y_train)
print("After SMOTE:", pd.Series(y_train_res).value_counts().to_dict())

# =========================
# RANDOM FOREST HYPERPARAMETER SEARCH
# =========================
rf = RandomForestClassifier(class_weight='balanced', random_state=RANDOM_STATE)
param_dist = {
    'n_estimators': [300, 500],
    'max_depth': [30, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

search = RandomizedSearchCV(
    rf, param_distributions=param_dist,
    n_iter=10, cv=3, scoring='roc_auc',
    random_state=RANDOM_STATE, n_jobs=-1, verbose=1
)
search.fit(X_train_res, y_train_res)
best_rf = search.best_estimator_
print("Best RF params:", search.best_params_)

# =========================
# CALIBRATE PROBABILITIES
# =========================
cal = CalibratedClassifierCV(best_rf, cv=3, method='isotonic')
cal.fit(X_train_res, y_train_res)
print("Calibrated classifier trained.")

# =========================
# EVALUATE TEST SET
# =========================
probs_test = cal.predict_proba(X_test_fp)
classes = cal.classes_.tolist()
active_idx = classes.index('active') if 'active' in classes else np.argmin(np.unique(y, return_counts=True)[1])
y_test_bin = np.array([1 if lab=='active' else 0 for lab in y_test])
y_proba_active = probs_test[:, active_idx]

prec, rec, thresh = precision_recall_curve(y_test_bin, y_proba_active)
fscore = (2 * prec * rec) / (prec + rec + 1e-12)
best_thresh = thresh[np.argmax(fscore)]
print("Optimal threshold (PR-F1):", best_thresh)
print("ROC-AUC:", roc_auc_score(y_test_bin, y_proba_active))
print("PR-AUC:", average_precision_score(y_test_bin, y_proba_active))

# =========================
# SAVE TRAINING ACTIVES
# =========================
train_actives = train_df[train_df['bioactivity_class']=='active']
active_canon_smis = train_actives['canonical_smiles'].tolist()
active_rdkit_fps = train_actives['fp'].tolist()

# =========================
# SAVE MODEL FOR APP
# =========================
# The app expects a dictionary with specific keys.
# Let's collect all the necessary data.
model_to_save = {
    'model': cal,
    'threshold': float(best_thresh),
    'fp_radius': FP_RADIUS,
    'fp_nbits': FP_NBITS,
    'active_canonical_smiles': active_canon_smis,
    # 'active_rdkit_fps' is not used by the app, so it can be omitted.
}

joblib.dump(model_to_save, "rf_fp_model_pro.pkl")
print("✅ Saved rf_fp_model_pro.pkl with calibrated RF and active fingerprints.")


In [3]:
# =========================
# SIMPLE SIMILARITY CHECK
# =========================

import joblib
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import Draw

# Model load karo
model_data = joblib.load("rf_fp_model_pro.pkl")

# Aapka molecule
test_smiles = "CCOC(=O)C1=CC2=CN=C(N=C2N1C)C1=CC=CC(=C1)C(F)(F)F"
test_mol = Chem.MolFromSmiles(test_smiles)

# Similarity check karo known active molecules se
print("Aapke molecule ki similarity known active molecules se:\n")

your_fp = AllChem.GetMorganFingerprint(test_mol, 2)
similarities = []

for i, active_smi in enumerate(model_data['active_canonical_smiles'][:5]):  # Pehle 5 active
    active_mol = Chem.MolFromSmiles(active_smi)
    active_fp = AllChem.GetMorganFingerprint(active_mol, 2)
    similarity = DataStructs.TanimotoSimilarity(your_fp, active_fp)
    similarities.append(similarity)
    
    print(f"Active Molecule {i+1}: {similarity:.3f} similarity")

print(f"\nAverage similarity: {np.mean(similarities):.3f}")
print(f"Maximum similarity: {np.max(similarities):.3f}")

# Molecule dikhao
print("\nAapka molecule structure:")
img = Draw.MolToImage(test_mol, size=(400, 300))
img.save("your_molecule.png")
print("Image saved as 'your_molecule.png'")

Aapke molecule ki similarity known active molecules se:

Active Molecule 1: 0.163 similarity
Active Molecule 2: 0.194 similarity
Active Molecule 3: 0.105 similarity
Active Molecule 4: 0.187 similarity
Active Molecule 5: 0.190 similarity

Average similarity: 0.168
Maximum similarity: 0.194

Aapka molecule structure:


[11:51:34] DEPRECATION WARNING: please use MorganGenerator
[11:51:34] DEPRECATION WARNING: please use MorganGenerator
[11:51:34] DEPRECATION WARNING: please use MorganGenerator
[11:51:34] DEPRECATION WARNING: please use MorganGenerator
[11:51:34] DEPRECATION WARNING: please use MorganGenerator
[11:51:34] DEPRECATION WARNING: please use MorganGenerator


Image saved as 'your_molecule.png'


In [1]:
"""
Expert revised end-to-end script for training a Random Forest on Morgan fingerprints
with careful handling of labels, oversampling, calibration, evaluation, and export.

Features:
- Robust SMILES parsing and canonicalization with logging
- Binary label mapping (active=1, others=0)
- Morgan fingerprint generation (nBits configurable)
- Preserves binary nature using RandomOverSampler (option to try SMOTE for experiments)
- RandomizedSearchCV with scoring='average_precision' (PR-AUC) for imbalanced data
- CalibratedClassifierCV (sigmoid by default) for well-behaved probabilities
- Proper PR-threshold selection (alignment of thresholds and precision/recall)
- Tanimoto similarity diagnostics between test compounds and training actives
- Saves model artifact with numpy fingerprints and canonical SMILES

Usage:
- Place this script in the same folder as your CSV (default name: Descriptors_Specified_SmallGTPases.csv)
- Run: python rf_fp_model_pro_expert.py

"""

import logging
import sys
import os
from typing import List, Optional, Tuple

import numpy as np
import pandas as pd
import joblib
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import (precision_recall_curve, roc_auc_score,
                             average_precision_score, classification_report,
                             confusion_matrix)
from imblearn.over_sampling import RandomOverSampler

# -------------------------
# CONFIG
# -------------------------
FP_RADIUS = 2
FP_NBITS = 2048
RANDOM_STATE = 42
CSV_PATH = "Descriptors_Specified_SmallGTPases.csv"
OUTPUT_MODEL = "rf_fp_model_pro_expert.pkl"
TEST_SIZE = 0.2
CV_FOLDS = 3
N_ITER_SEARCH = 20
N_JOBS = -1
CALIBRATION_METHOD = 'sigmoid'  # 'sigmoid' or 'isotonic' (use isotonic only with ample calibration data)
USE_SMOTE = False  # leave False for fingerprints; set True only for experiments

# -------------------------
# LOGGING
# -------------------------
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s: %(message)s")
logger = logging.getLogger(__name__)

# -------------------------
# Helpers - RDKit and fingerprints
# -------------------------

def mol_from_smiles(smiles: str) -> Optional[Chem.Mol]:
    """Parse SMILES to RDKit Mol. Returns None on failure (logs reason)."""
    if not isinstance(smiles, str):
        return None
    try:
        m = Chem.MolFromSmiles(smiles, sanitize=True)
        return m
    except Exception as e:
        logger.debug(f"SMILES parse failed for {smiles}: {e}")
        return None


def canonical_smiles(smiles: str) -> Optional[str]:
    m = mol_from_smiles(smiles)
    if m is None:
        return None
    try:
        return Chem.MolToSmiles(m, canonical=True)
    except Exception as e:
        logger.debug(f"Canonicalize failed for {smiles}: {e}")
        return None


def mol_to_fp(mol: Chem.Mol, radius: int = FP_RADIUS, nBits: int = FP_NBITS):
    """Return RDKit ExplicitBitVect Morgan fingerprint for a molecule."""
    return AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=nBits)


def fp_to_numpy(fp) -> np.ndarray:
    arr = np.zeros((FP_NBITS,), dtype=np.uint8)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr


def fp_from_smiles(smiles: str) -> Optional[np.ndarray]:
    m = mol_from_smiles(smiles)
    if m is None:
        return None
    fp = mol_to_fp(m)
    return fp_to_numpy(fp)

# -------------------------
# Similarity utilities
# -------------------------

def rdkit_fp_from_numpy(arr: np.ndarray):
    """Convert numpy 0/1 array back to RDKit ExplicitBitVect for Tanimoto computations."""
    bv = DataStructs.ExplicitBitVect(FP_NBITS)
    for i, v in enumerate(arr):
        if int(v):
            bv.SetBit(i)
    return bv


def max_tanimoto_to_fps(target_fp_arr: np.ndarray, fps_arr_list: List[np.ndarray]) -> float:
    if len(fps_arr_list) == 0:
        return 0.0
    t_fp = rdkit_fp_from_numpy(target_fp_arr)
    sims = []
    for a in fps_arr_list:
        a_fp = rdkit_fp_from_numpy(a)
        sims.append(DataStructs.TanimotoSimilarity(t_fp, a_fp))
    return float(np.max(sims))

# -------------------------
# Load & prepare data
# -------------------------

def load_and_prepare(csv_path: str) -> Tuple[pd.DataFrame, np.ndarray, np.ndarray]:
    if not os.path.exists(csv_path):
        logger.error(f"CSV not found: {csv_path}")
        sys.exit(1)

    df = pd.read_csv(csv_path)
    logger.info(f"Original dataset shape: {df.shape}")

    # Ensure bioactivity_class present
    if 'bioactivity_class' not in df.columns:
        logger.error("CSV must contain 'bioactivity_class' column")
        sys.exit(1)

    # Fill canonical_smiles from 'smiles' if missing, canonicalize everything
    if 'canonical_smiles' not in df.columns:
        df['canonical_smiles'] = None

    df['canonical_smiles'] = df['canonical_smiles'].fillna(df.get('smiles', np.nan))
    df['canonical_smiles'] = df['canonical_smiles'].apply(lambda s: canonical_smiles(s) if pd.notna(s) else None)
    before = df.shape[0]
    df = df.dropna(subset=['canonical_smiles']).reset_index(drop=True)
    logger.info(f"After canonicalization and dropping invalid SMILES: {df.shape} (dropped {before - df.shape[0]})")

    # Map labels to binary: active -> 1, everything else -> 0
    df['bioactivity_class'] = df['bioactivity_class'].astype(str)
    df['y_bin'] = (df['bioactivity_class'].str.lower() == 'active').astype(int)
    logger.info(f"Label distribution:\n{df['bioactivity_class'].value_counts().to_dict()}\nBinary counts: {df['y_bin'].value_counts().to_dict()}")

    # Compute fingerprints (numpy arrays)
    fps = []
    bad = 0
    for smi in df['canonical_smiles']:
        arr = fp_from_smiles(smi)
        if arr is None:
            fps.append(None)
            bad += 1
        else:
            fps.append(arr)
    if bad:
        logger.info(f"Failed to compute fingerprints for {bad} molecules")
    df['fp_arr'] = fps
    df = df.dropna(subset=['fp_arr']).reset_index(drop=True)
    logger.info(f"After removing molecules without fingerprints: {df.shape}")

    X = np.vstack(df['fp_arr'].values)
    y = df['y_bin'].values
    return df, X, y

# -------------------------
# Train / evaluate flow
# -------------------------

def train_and_evaluate(df: pd.DataFrame, X: np.ndarray, y: np.ndarray):
    # split
    X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
        X, y, df.index, test_size=TEST_SIZE, stratify=y, random_state=RANDOM_STATE
    )
    train_df = df.loc[idx_train].reset_index(drop=True)
    test_df = df.loc[idx_test].reset_index(drop=True)

    # Oversample minority class with RandomOverSampler to preserve binary fingerprints
    ros = RandomOverSampler(random_state=RANDOM_STATE)
    X_train_res, y_train_res = ros.fit_resample(X_train, y_train)
    logger.info(f"After RandomOverSampler: {pd.Series(y_train_res).value_counts().to_dict()}")

    # Random Forest + randomized search
    rf = RandomForestClassifier(class_weight='balanced', random_state=RANDOM_STATE, n_jobs=1)
    param_dist = {
        'n_estimators': [200, 300, 500],
        'max_depth': [20, 30, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2', None]
    }

    search = RandomizedSearchCV(
        rf, param_distributions=param_dist,
        n_iter=N_ITER_SEARCH, cv=StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE),
        scoring='average_precision', random_state=RANDOM_STATE, n_jobs=N_JOBS, verbose=1
    )
    search.fit(X_train_res, y_train_res)
    best_rf = search.best_estimator_
    logger.info(f"Best RF params: {search.best_params_}")

    # Calibrate
    cal = CalibratedClassifierCV(best_rf, cv=CV_FOLDS, method=CALIBRATION_METHOD)
    cal.fit(X_train_res, y_train_res)
    logger.info("Calibrated classifier trained.")

    # Predict probabilities on test
    probs_test = cal.predict_proba(X_test)
    classes = list(cal.classes_)
    # positive index: prefer numeric 1 -> otherwise 'active'
    if 1 in classes:
        pos_idx = classes.index(1)
    elif 'active' in classes:
        pos_idx = classes.index('active')
    else:
        pos_idx = 1 if len(classes) > 1 else 0
    y_proba = probs_test[:, pos_idx]

    # Metrics
    ap = average_precision_score(y_test, y_proba)
    try:
        roc = roc_auc_score(y_test, y_proba)
    except Exception:
        roc = float('nan')
    prec, rec, thresholds = precision_recall_curve(y_test, y_proba)

    # Choose PR-F1 optimal threshold (align indices)
    if len(thresholds) > 0:
        fscore = (2 * prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-12)
        best_idx = int(np.nanargmax(fscore))
        best_thresh = float(thresholds[best_idx])
        best_f1 = float(fscore[best_idx])
    else:
        best_thresh = 0.5
        best_f1 = None

    logger.info(f"Test AP (PR-AUC / average precision): {ap:.4f}")
    logger.info(f"Test ROC-AUC: {roc}")
    logger.info(f"Best PR-F1 threshold: {best_thresh} (F1: {best_f1})")

    # Predictions at chosen threshold
    y_pred_at_thresh = (y_proba >= best_thresh).astype(int)

    logger.info("Classification report (at chosen threshold):\n" + classification_report(y_test, y_pred_at_thresh))
    logger.info("Confusion matrix:\n" + str(confusion_matrix(y_test, y_pred_at_thresh)))

    # Save training actives and their fingerprints
    train_actives = train_df[train_df['y_bin'] == 1]
    active_smiles = train_actives['canonical_smiles'].tolist()
    active_fps_np = train_actives['fp_arr'].tolist()

    # Tanimoto similarity diagnostics: for each test molecule, max similarity to training actives
    train_active_fps_np = active_fps_np
    if len(train_active_fps_np) > 0:
        max_sims = [max_tanimoto_to_fps(arr, train_active_fps_np) for arr in test_df['fp_arr'].values]
        test_df['max_sim_to_train_actives'] = max_sims
        logger.info(f"Test max-similarity to train actives (describe):\n{pd.Series(max_sims).describe()}" )
    else:
        test_df['max_sim_to_train_actives'] = np.nan
        logger.info("No training actives available for similarity diagnostics.")

    # Package model artifact
    model_data = {
        'model': cal,
        'threshold': best_thresh,
        'fp_radius': FP_RADIUS,
        'fp_nbits': FP_NBITS,
        'active_canonical_smiles': active_smiles,
        'active_fp_numpy': active_fps_np,
        'feature_info': {
            'type': 'morgan', 'radius': FP_RADIUS, 'nBits': FP_NBITS
        }
    }

    joblib.dump(model_data, OUTPUT_MODEL)
    logger.info(f"Saved model artifact to {OUTPUT_MODEL}")

    # Also return key objects for further inspection
    results = {
        'search': search,
        'calibrated_model': cal,
        'best_threshold': best_thresh,
        'y_test': y_test,
        'y_proba': y_proba,
        'test_df': test_df,
        'train_df': train_df,
        'metrics': {'average_precision': ap, 'roc_auc': roc, 'best_f1': best_f1}
    }
    return results

# -------------------------
# MAIN
# -------------------------
if __name__ == '__main__':
    df, X, y = load_and_prepare(CSV_PATH)
    results = train_and_evaluate(df, X, y)
    logger.info("Training + evaluation complete.")

    # Quick print of key metrics
    m = results['metrics']
    logger.info(f"Metrics summary: AP={m['average_precision']:.4f} ROC-AUC={m['roc_auc']} best_f1={m['best_f1']}")

    # Optionally save test results
    try:
        results['test_df'][['canonical_smiles', 'max_sim_to_train_actives']].to_csv('test_similarity_report.csv', index=False)
        logger.info("Saved test_similarity_report.csv")
    except Exception:
        logger.debug("Could not save test similarity report.")

    logger.info("Done.")


2025-09-11 19:19:09,339 INFO: Original dataset shape: (7686, 21)
2025-09-11 19:19:51,862 INFO: After canonicalization and dropping invalid SMILES: (7686, 21) (dropped 0)
2025-09-11 19:19:51,890 INFO: Label distribution:
{'inactive': 6296, 'active': 1390}
Binary counts: {0: 6296, 1: 1390}
[19:19:51] DEPRECATION WARNING: please use MorganGenerator
[19:19:51] DEPRECATION WARNING: please use MorganGenerator
[19:19:51] DEPRECATION WARNING: please use MorganGenerator
[19:19:51] DEPRECATION WARNING: please use MorganGenerator
[19:19:51] DEPRECATION WARNING: please use MorganGenerator
[19:19:51] DEPRECATION WARNING: please use MorganGenerator
[19:19:52] DEPRECATION WARNING: please use MorganGenerator
[19:19:52] DEPRECATION WARNING: please use MorganGenerator
[19:19:52] DEPRECATION WARNING: please use MorganGenerator
[19:19:52] DEPRECATION WARNING: please use MorganGenerator
[19:19:52] DEPRECATION WARNING: please use MorganGenerator
[19:19:52] DEPRECATION WARNING: please use MorganGenerator
[19

Fitting 3 folds for each of 20 candidates, totalling 60 fits


KeyboardInterrupt: 

In [4]:
# qsar_small_gtpases_model.py
# QSAR pipeline: build RandomForest classifier for Small GTPases bioactivity
# Save model and helper function to predict new SMILES with AD checks.
# Designed for laptop (toggle FINAL_MODE for heavier runs).

import os
import logging
import numpy as np
import pandas as pd
import joblib
from collections import Counter
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem.Scaffolds import MurckoScaffold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import average_precision_score, roc_auc_score, precision_recall_curve, classification_report, confusion_matrix
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import RandomOverSampler

# -------------------------
# CONFIG
# -------------------------
CSV_PATH = "Descriptors_Specified_SmallGTPases.csv"
SMILES_COL = "canonical_smiles"
LABEL_COL = "bioactivity_class"   # 'active' / 'inactive'
PIC50_COL = "pIC50"               # numeric potency (optional)
OUTPUT_MODEL = "qsar_small_gtpases_artifact.joblib"

FINAL_MODE = False  # False = debug (fast). True = final (heavier)
if FINAL_MODE:
    FP_NBITS = 2048
    RAND_ITERS = 20
    CV_FOLDS = 3
    RF_N_EST = [200, 300]
    N_JOBS = 2
else:
    FP_NBITS = 1024
    RAND_ITERS = 6
    CV_FOLDS = 2
    RF_N_EST = [50, 100]
    N_JOBS = 1

FP_RADIUS = 2
TEST_SIZE = 0.2
RANDOM_STATE = 42
MAX_TANIMOTO_AD = 0.4  # below this considered structurally outside AD
LEVERAGE_MULTIPLIER = 3  # for descriptor leverage AD

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s: %(message)s")
logger = logging.getLogger(__name__)

# -------------------------
# Helpers (RDKit + utils)
# -------------------------
def largest_fragment_smiles(smi):
    try:
        m = Chem.MolFromSmiles(smi)
        if m is None: return None
        frags = Chem.GetMolFrags(m, asMols=True)
        largest = max(frags, key=lambda fm: fm.GetNumHeavyAtoms())
        return Chem.MolToSmiles(largest, canonical=True)
    except Exception:
        return None

def canonicalize_smiles(smi):
    try:
        m = Chem.MolFromSmiles(smi)
        return Chem.MolToSmiles(m, canonical=True) if m else None
    except Exception:
        return None

def mol_to_fp_numpy(mol, radius=FP_RADIUS, nBits=FP_NBITS):
    if mol is None: return None
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=nBits)
    arr = np.zeros((nBits,), dtype=np.uint8)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

def rdkit_fp_from_numpy(arr):
    bv = DataStructs.ExplicitBitVect(len(arr))
    for i,v in enumerate(arr):
        if int(v): bv.SetBit(i)
    return bv

def tanimoto(a_arr, b_arr):
    return DataStructs.TanimotoSimilarity(rdkit_fp_from_numpy(a_arr), rdkit_fp_from_numpy(b_arr))

# -------------------------
# Load + basic checks
# -------------------------
if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f"CSV not found at {CSV_PATH}")

logger.info("Loading CSV...")
df = pd.read_csv(CSV_PATH)
required = {SMILES_COL, LABEL_COL}
if not required.issubset(set(df.columns)):
    raise ValueError(f"CSV must contain columns: {required}. Found: {df.columns.tolist()}")

# -------------------------
# SMILES curation & dedup
# -------------------------
logger.info("Canonicalizing and keeping largest fragment for each SMILES...")
df['smiles_clean'] = df[SMILES_COL].astype(str).apply(largest_fragment_smiles)
before = len(df)
df = df.dropna(subset=['smiles_clean']).reset_index(drop=True)
logger.info(f"Dropped {before - len(df)} rows due to invalid SMILES")

# If duplicates by canonical SMILES, aggregate pIC50 (if available) and majority class
if PIC50_COL in df.columns:
    agg_funcs = {PIC50_COL: 'median', LABEL_COL: (lambda x: Counter(x).most_common(1)[0][0])}
    df = df.groupby('smiles_clean', as_index=False).agg(agg_funcs).reset_index(drop=True)
    logger.info("Aggregated duplicates by median pIC50 and majority class.")
else:
    df = df.drop_duplicates(subset=['smiles_clean']).reset_index(drop=True)
    logger.info("Dropped exact duplicate SMILES (no pIC50 column present).")

# -------------------------
# Compute fingerprint arrays
# -------------------------
logger.info("Computing Morgan fingerprints...")
fps = []
bad = 0
for smi in df['smiles_clean']:
    m = Chem.MolFromSmiles(smi)
    arr = mol_to_fp_numpy(m)
    if arr is None:
        fps.append(None); bad += 1
    else:
        fps.append(arr)
if bad:
    logger.info(f"Failed to compute fingerprints for {bad} molecules")
df['fp_arr'] = fps
df = df.dropna(subset=['fp_arr']).reset_index(drop=True)
logger.info(f"Dataset after fingerprinting: {len(df)} rows")

# -------------------------
# Descriptor selection & cleaning
# -------------------------
# Choose numeric columns from dataset excluding SMILES and label/pIC50/fp
exclude = {'smiles_clean', SMILES_COL, LABEL_COL, PIC50_COL, 'fp_arr'}
numeric_cols = [c for c in df.columns if c not in exclude and pd.api.types.is_numeric_dtype(df[c])]
logger.info(f"Numeric descriptor columns found: {numeric_cols}")

# Remove near-constant descriptors
if numeric_cols:
    desc_df = df[numeric_cols].copy()
    vt = VarianceThreshold(threshold=1e-5)
    try:
        vt.fit(desc_df.fillna(0))
        keep = np.array(numeric_cols)[vt.get_support()].tolist()
        if set(keep) != set(numeric_cols):
            logger.info(f"Dropping near-constant descriptors: {set(numeric_cols)-set(keep)}")
        numeric_cols = keep
    except Exception as e:
        logger.debug("VarianceThreshold failed: " + str(e))

# Clip extreme outliers for numeric descriptors (optionally)
for c in numeric_cols:
    q_low, q_high = df[c].quantile(0.01), df[c].quantile(0.99)
    df[c] = df[c].clip(lower=q_low, upper=q_high)

# Build feature matrices
X_fp = np.vstack(df['fp_arr'].values)  # shape (n, FP_NBITS)
X_desc = df[numeric_cols].fillna(0).values if numeric_cols else np.zeros((len(df), 0))
y = (df[LABEL_COL].astype(str).str.lower() == 'active').astype(int).values

logger.info(f"Final feature sizes: fingerprints {X_fp.shape}, descriptors {X_desc.shape}, labels {Counter(y)}")

# -------------------------
# Scaffold split (Murcko)
# -------------------------
logger.info("Performing Murcko scaffold split for realistic test set...")
def scaffold_of_smiles(smi):
    mol = Chem.MolFromSmiles(smi)
    if mol is None: return None
    try:
        sc = MurckoScaffold.GetScaffoldForMol(mol)
        return Chem.MolToSmiles(sc, isomericSmiles=False)
    except Exception:
        return None

df['scaffold'] = df['smiles_clean'].apply(scaffold_of_smiles)
groups = df.groupby('scaffold').indices
scaffolds = list(groups.keys())
rng = np.random.RandomState(RANDOM_STATE)
rng.shuffle(scaffolds)

test_idx, train_idx = [], []
n_test_target = int(len(df) * TEST_SIZE)
count = 0
for sc in scaffolds:
    idxs = list(groups[sc])
    if count < n_test_target:
        test_idx.extend(idxs); count += len(idxs)
    else:
        train_idx.extend(idxs)

train_df = df.loc[train_idx].reset_index(drop=True)
test_df = df.loc[test_idx].reset_index(drop=True)
logger.info(f"Train/test sizes (scaffold split): {len(train_df)} / {len(test_df)}")

# Build train/test feature arrays
X_fp_train = np.vstack(train_df['fp_arr'].values)
X_desc_train = train_df[numeric_cols].fillna(0).values if numeric_cols else np.zeros((len(train_df),0))
y_train = (train_df[LABEL_COL].astype(str).str.lower() == 'active').astype(int).values

X_fp_test = np.vstack(test_df['fp_arr'].values)
X_desc_test = test_df[numeric_cols].fillna(0).values if numeric_cols else np.zeros((len(test_df),0))
y_test = (test_df[LABEL_COL].astype(str).str.lower() == 'active').astype(int).values

# -------------------------
# Oversample minority (train only)
# -------------------------
logger.info("Oversampling minority class with RandomOverSampler...")
X_train_comb = np.hstack([X_fp_train, X_desc_train])
ros = RandomOverSampler(random_state=RANDOM_STATE)
X_train_res, y_train_res = ros.fit_resample(X_train_comb, y_train)
fp_len = X_fp_train.shape[1]
X_fp_train_res = X_train_res[:, :fp_len]
X_desc_train_res = X_train_res[:, fp_len:]

# -------------------------
# Hyperparam search (RandomForest) - conservative sizes for laptop
# -------------------------
logger.info("Starting RandomizedSearchCV for RandomForest (this is the slowest step)...")
base = RandomForestClassifier(class_weight='balanced', random_state=RANDOM_STATE, n_jobs=1)

param_dist = {
    'n_estimators': RF_N_EST,
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt']
}

search = RandomizedSearchCV(base,
                            param_distributions=param_dist,
                            n_iter=RAND_ITERS,
                            cv=StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE),
                            scoring='average_precision',
                            random_state=RANDOM_STATE,
                            n_jobs=N_JOBS,
                            verbose=2)

X_train_comb_res = np.hstack([X_fp_train_res, X_desc_train_res])
search.fit(X_train_comb_res, y_train_res)
best_rf = search.best_estimator_
logger.info(f"Best RF params: {search.best_params_}")

# -------------------------
# Calibration
# -------------------------
logger.info("Calibrating probabilities with sigmoid (CalibratedClassifierCV)...")
cal = CalibratedClassifierCV(best_rf, method='sigmoid', cv=CV_FOLDS)
cal.fit(X_train_comb_res, y_train_res)

# -------------------------
# Evaluate on test set
# -------------------------
X_test_comb = np.hstack([X_fp_test, X_desc_test])
y_proba = cal.predict_proba(X_test_comb)[:, 1]

ap = average_precision_score(y_test, y_proba)
roc = roc_auc_score(y_test, y_proba) if len(np.unique(y_test))>1 else float('nan')
precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
if len(thresholds)>0:
    fscore = (2 * precision[:-1] * recall[:-1]) / (precision[:-1] + recall[:-1] + 1e-12)
    best_idx = int(np.nanargmax(fscore))
    best_threshold = float(thresholds[best_idx])
else:
    best_threshold = 0.5

y_pred = (y_proba >= best_threshold).astype(int)

logger.info(f"Test AP (average precision / PR-AUC): {ap:.4f}")
logger.info(f"Test ROC-AUC: {roc:.4f}")
print("\nClassification report (at PR-optimal threshold):")
print(classification_report(y_test, y_pred, digits=4))
print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred))

# -------------------------
# Applicability Domain (AD)
# - structural AD: max Tanimoto to training actives
# - descriptor AD: leverage using training descriptor matrix
# -------------------------
logger.info("Computing applicability domain diagnostics...")

# training active fps numpy
train_active_fps = [arr for arr, lab in zip(X_fp_train_res, y_train_res) if lab==1]

# compute test max Tanimoto to train actives
test_max_sims = [max(tanimoto(arr, a) for a in train_active_fps) if len(train_active_fps)>0 else 0.0 for arr in X_fp_test]
test_df['max_sim_to_train_actives'] = test_max_sims

# descriptor leverage (on training descriptors)
if X_desc_train_res.shape[1] > 0:
    Xd = np.hstack([np.ones((X_desc_train_res.shape[0],1)), X_desc_train_res])
    M = np.linalg.pinv(Xd.T.dot(Xd))
    def leverage_of(x_desc):
        x0 = np.hstack([1.0, x_desc])
        return float(x0.dot(M).dot(x0.T))
    leverages = [leverage_of(x) for x in X_desc_test]
    h_cut = LEVERAGE_MULTIPLIER * (X_desc_train_res.shape[1]+1) / X_desc_train_res.shape[0]
    test_df['leverage'] = leverages
    test_df['in_leverage_ad'] = test_df['leverage'] <= h_cut
else:
    test_df['leverage'] = np.nan
    test_df['in_leverage_ad'] = True

test_df['in_structural_ad'] = test_df['max_sim_to_train_actives'] >= MAX_TANIMOTO_AD
test_df['in_applicability_domain'] = test_df['in_structural_ad'] & test_df['in_leverage_ad']

logger.info("AD summary for test set:")
logger.info(test_df[['max_sim_to_train_actives','leverage','in_applicability_domain']].describe().to_string())

# -------------------------
# Save artifact (model + metadata)
# -------------------------
artifact = {
    'model': cal,
    'fp_radius': FP_RADIUS,
    'fp_nbits': FP_NBITS,
    'descriptor_columns': numeric_cols,
    'best_params': search.best_params_,
    'best_threshold': best_threshold,
    'train_active_fps': train_active_fps,
    'X_desc_train_res': X_desc_train_res,
    'Xd_pinv': (M if X_desc_train_res.shape[1]>0 else None),
}

joblib.dump(artifact, OUTPUT_MODEL, compress=3)
logger.info(f"Saved model artifact to {OUTPUT_MODEL}")

# -------------------------
# Helper: predict new SMILES with AD check
# -------------------------
def predict_smiles(smiles_list):
    """
    Input: list of SMILES strings
    Returns: list of dicts: {smiles, proba_active, pred_label, in_AD (bool), max_sim}
    """
    results = []
    for smi in smiles_list:
        smi2 = largest_fragment_smiles(smi)
        mol = Chem.MolFromSmiles(smi2) if smi2 else None
        fp = mol_to_fp_numpy(mol)
        desc = np.array([0.0]*len(numeric_cols)) if len(numeric_cols)==0 else np.array([0.0]*len(numeric_cols))
        if fp is None:
            results.append({'smiles': smi, 'error': 'invalid_smiles'})
            continue
        x_comb = np.hstack([fp, desc])
        proba = artifact['model'].predict_proba(x_comb.reshape(1,-1))[:,1][0]
        # AD checks
        max_sim = max(tanimoto(fp, a) for a in artifact['train_active_fps']) if len(artifact['train_active_fps'])>0 else 0.0
        in_struct_ad = max_sim >= MAX_TANIMOTO_AD
        if artifact['Xd_pinv'] is not None:
            # leverage
            xdesc = desc
            M_local = artifact['Xd_pinv']
            x0 = np.hstack([1.0, xdesc])
            h0 = float(x0.dot(M_local).dot(x0.T))
            in_lev = h0 <= LEVERAGE_MULTIPLIER * (len(xdesc)+1) / artifact['X_desc_train_res'].shape[0]
        else:
            in_lev = True
        in_ad = in_struct_ad and in_lev
        pred = int(proba >= artifact['best_threshold'])
        results.append({'smiles': smi, 'proba_active': float(proba), 'pred_label': pred, 'in_AD': bool(in_ad), 'max_sim': float(max_sim)})
    return results

logger.info("You can now call predict_smiles([...]) on new SMILES to get predictions + AD flags.")


2025-09-12 09:33:36,022 INFO: Loading CSV...
2025-09-12 09:33:36,082 INFO: Canonicalizing and keeping largest fragment for each SMILES...
2025-09-12 09:33:46,270 INFO: Dropped 0 rows due to invalid SMILES
2025-09-12 09:33:46,554 INFO: Aggregated duplicates by median pIC50 and majority class.
2025-09-12 09:33:46,555 INFO: Computing Morgan fingerprints...
[09:33:46] DEPRECATION WARNING: please use MorganGenerator
[09:33:46] DEPRECATION WARNING: please use MorganGenerator
[09:33:46] DEPRECATION WARNING: please use MorganGenerator
[09:33:46] DEPRECATION WARNING: please use MorganGenerator
[09:33:46] DEPRECATION WARNING: please use MorganGenerator
[09:33:46] DEPRECATION WARNING: please use MorganGenerator
[09:33:46] DEPRECATION WARNING: please use MorganGenerator
[09:33:46] DEPRECATION WARNING: please use MorganGenerator
[09:33:46] DEPRECATION WARNING: please use MorganGenerator
[09:33:46] DEPRECATION WARNING: please use MorganGenerator
[09:33:46] DEPRECATION WARNING: please use MorganGener

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   1.2s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   2.1s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   2.0s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   2.3s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   2.3s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   2.3s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   2.3s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=50; tot

2025-09-12 09:34:37,385 INFO: Best RF params: {'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': None}
2025-09-12 09:34:37,386 INFO: Calibrating probabilities with sigmoid (CalibratedClassifierCV)...
2025-09-12 09:34:42,817 INFO: Test AP (average precision / PR-AUC): 0.8924
2025-09-12 09:34:42,817 INFO: Test ROC-AUC: 0.9711
2025-09-12 09:34:42,833 INFO: Computing applicability domain diagnostics...



Classification report (at PR-optimal threshold):
              precision    recall  f1-score   support

           0     0.9671    0.9625    0.9648      1253
           1     0.8374    0.8551    0.8462       283

    accuracy                         0.9427      1536
   macro avg     0.9022    0.9088    0.9055      1536
weighted avg     0.9432    0.9427    0.9429      1536

Confusion matrix:
[[1206   47]
 [  41  242]]


KeyboardInterrupt: 

In [5]:
# train_fp_pro_production.py
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import joblib
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import roc_auc_score, average_precision_score
from imblearn.over_sampling import SMOTE

# CONFIG
FP_RADIUS = 2
FP_NBITS = 2048
RANDOM_STATE = 42

# HELPERS
def mol_from_smiles(smiles):
    try:
        return Chem.MolFromSmiles(smiles)
    except:
        return None

def canonical_smiles(smiles):
    m = mol_from_smiles(smiles)
    return Chem.MolToSmiles(m, canonical=True) if m else None

def mol_to_fp(mol):
    return AllChem.GetMorganFingerprintAsBitVect(mol, FP_RADIUS, nBits=FP_NBITS)

def fp_to_numpy(fp):
    arr = np.zeros((FP_NBITS,), dtype=np.uint8)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

def tanimoto(fp_a, fp_b):
    """Tanimoto between two RDKit ExplicitBitVect objects OR numpy arrays"""
    if isinstance(fp_a, np.ndarray) and isinstance(fp_b, np.ndarray):
        # both numpy arrays of 0/1
        a = fp_a.astype(bool)
        b = fp_b.astype(bool)
        inter = np.logical_and(a,b).sum()
        union = np.logical_or(a,b).sum()
        return float(inter) / float(union) if union>0 else 0.0
    else:
        # RDKit bit vects
        return DataStructs.TanimotoSimilarity(fp_a, fp_b)

# LOAD DATA
df = pd.read_csv("Descriptors_Specified_SmallGTPases.csv")
print("Original shape:", df.shape)

# Normalize labels
df['bioactivity_class'] = df['bioactivity_class'].astype(str).str.lower().str.strip()
df = df.replace({'bioactivity_class': {'act':'active','inact':'inactive'}})

# Keep only active/inactive (drop intermediates or unknown)
df = df[df['bioactivity_class'].isin(['active','inactive'])].copy()
print("After keeping binary labels:", df.shape)
print(df['bioactivity_class'].value_counts())

# Fill and canonicalize smiles
df['canonical_smiles'] = df['canonical_smiles'].fillna(df.get('smiles', np.nan))
df['canonical_smiles'] = df['canonical_smiles'].apply(lambda s: canonical_smiles(s) if pd.notna(s) else None)
df = df.dropna(subset=['canonical_smiles']).reset_index(drop=True)
print("After canonicalization:", df.shape)

# Compute fingerprints and numpy versions
rdkit_fps = []
np_fps = []
bad_idx = []
for i, smi in enumerate(df['canonical_smiles']):
    m = mol_from_smiles(smi)
    if m:
        fp = mol_to_fp(m)
        rdkit_fps.append(fp)
        np_fps.append(fp_to_numpy(fp))
    else:
        bad_idx.append(i)

if bad_idx:
    df = df.drop(df.index[bad_idx]).reset_index(drop=True)
    print("Dropped invalid SMILES indices:", bad_idx)

df['rdkit_fp'] = rdkit_fps
df['fp_arr'] = np_fps
print("After fingerprints:", df.shape)

# PREPARE X and y
X_fp = np.vstack(df['fp_arr'].values)
y = df['bioactivity_class'].values

# TRAIN-TEST SPLIT (stratified)
X_train_fp, X_test_fp, y_train, y_test, idx_train, idx_test = train_test_split(
    X_fp, y, df.index, test_size=0.2, stratify=y, random_state=RANDOM_STATE
)
train_df = df.loc[idx_train].reset_index(drop=True)
test_df = df.loc[idx_test].reset_index(drop=True)
print("Train/test sizes:", train_df.shape, test_df.shape)

# SMOTE: only if both classes present and minority >1
unique, counts = np.unique(y_train, return_counts=True)
print("Train class counts before SMOTE:", dict(zip(unique, counts)))
do_smote = (len(unique) == 2) and (counts.min() > 1)
if do_smote:
    smote = SMOTE(random_state=RANDOM_STATE)
    X_train_res, y_train_res = smote.fit_resample(X_train_fp, y_train)
    print("After SMOTE:", pd.Series(y_train_res).value_counts().to_dict())
else:
    X_train_res, y_train_res = X_train_fp, y_train
    print("Skipped SMOTE (not enough minority samples)")

# MODEL SEARCH
rf = RandomForestClassifier(class_weight='balanced', random_state=RANDOM_STATE, n_jobs=-1)
param_dist = {
    'n_estimators': [200, 300, 500],
    'max_depth': [20, 30, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}
scoring = 'roc_auc' if len(np.unique(y_train_res))==2 else 'roc_auc_ovr'
search = RandomizedSearchCV(rf, param_distributions=param_dist, n_iter=12, cv=3,
                            scoring=scoring, random_state=RANDOM_STATE, n_jobs=-1, verbose=1)
search.fit(X_train_res, y_train_res)
best_rf = search.best_estimator_
print("Best params:", search.best_params_)

# CALIBRATE probabilities (sigmoid is safer for small datasets)
try:
    cal = CalibratedClassifierCV(best_rf, cv=3, method='isotonic')
    cal.fit(X_train_res, y_train_res)
    print("Isotonic calibration succeeded.")
except Exception as e:
    print("Isotonic failed, falling back to sigmoid. Err:", e)
    cal = CalibratedClassifierCV(best_rf, cv=3, method='sigmoid')
    cal.fit(X_train_res, y_train_res)
    print("Sigmoid calibration succeeded.")

# EVALUATE (report only; no threshold selection)
probs_test = cal.predict_proba(X_test_fp)
classes = cal.classes_.tolist()
if 'active' in classes:
    active_idx = classes.index('active')
else:
    # fallback: assume positive label is the one with fewer samples in training
    active_idx = 1 if classes[1] != classes[0] else 0

y_test_bin = np.array([1 if lab=='active' else 0 for lab in y_test])
y_proba_active = probs_test[:, active_idx]
print("ROC-AUC (test):", roc_auc_score(y_test_bin, y_proba_active))
print("PR-AUC (test):", average_precision_score(y_test_bin, y_proba_active))

# PREPARE training active fps for similarity checks
train_actives = train_df[train_df['bioactivity_class']=='active'].copy()
train_actives = train_actives.reset_index(drop=True)
train_active_np_fps = np.stack(train_actives['fp_arr'].values) if len(train_actives)>0 else np.zeros((0, FP_NBITS), dtype=np.uint8)
train_active_rdkit_fps = train_actives['rdkit_fp'].tolist()

# Save model info (no forced threshold)
model_data = {
    'model': cal,
    'fp_radius': FP_RADIUS,
    'fp_nbits': FP_NBITS,
    'train_active_canonical_smiles': train_actives['canonical_smiles'].tolist(),
    'train_active_fp_arr': train_active_np_fps,  # numpy arrays for fast Tanimoto
    'train_active_rdkit_fps': train_active_rdkit_fps, # optional
    'notes': 'No fixed decision threshold stored. Use prob + similarity for decisions.'
}

joblib.dump(model_data, "rf_fp_model_pro_no_threshold.pkl")
print("Saved rf_fp_model_pro_no_threshold.pkl")

# ----------------------------
# PREDICTION helper for web app (use this function in app)
# ----------------------------
def predict_smiles(smiles, model_data, top_k=5):
    """
    Returns a dictionary:
      - prob_active: calibrated probability that compound is active (0..1)
      - max_tanimoto: max Tanimoto similarity to any training active
      - top_matches: list of (smiles, tanimoto) for top_k most similar train actives
      - confidence: 'high'/'medium'/'low' (advisory only) based on similarity and prob
    """
    cal = model_data['model']
    train_arrs = model_data['train_active_fp_arr']
    mol = mol_from_smiles(smiles)
    if mol is None:
        return {'error': 'invalid_smiles'}
    fp = mol_to_fp(mol)
    arr = fp_to_numpy(fp).reshape(1, -1)
    # probability
    probs = cal.predict_proba(arr)
    classes = cal.classes_.tolist()
    if 'active' in classes:
        p_active = float(probs[0][classes.index('active')])
    else:
        # fallback
        p_active = float(probs[0].max())

    # similarity to train actives (numpy arr)
    max_t = 0.0
    top_list = []
    if train_arrs.shape[0] > 0:
        # compute Tanimotos vectorized
        # simple loop (fast enough for tens-hundreds); optimize if you have thousands
        sims = [tanimoto(arr.ravel(), train_arrs[i]) for i in range(train_arrs.shape[0])]
        sims = np.array(sims)
        order = np.argsort(sims)[::-1][:top_k]
        top_list = [(model_data['train_active_canonical_smiles'][i], float(sims[i])) for i in order]
        max_t = float(sims.max())

    # confidence advisory (no hard rule: just recommendation)
    if max_t >= 0.7 and p_active >= 0.7:
        conf = 'high'
    elif max_t >= 0.4 or p_active >= 0.5:
        conf = 'medium'
    else:
        conf = 'low'  # low similarity AND low probability

    return {
        'prob_active': p_active,
        'max_tanimoto': max_t,
        'top_matches': top_list,
        'confidence_advisory': conf
    }

# Example: quick local test
if __name__ == '__main__':
    m = model_data
    test_smi = test_df['canonical_smiles'].iloc[0]
    print("Test example:", test_smi, predict_smiles(test_smi, m))


Original shape: (7686, 21)
After keeping binary labels: (7686, 21)
bioactivity_class
inactive    6296
active      1390
Name: count, dtype: int64
After canonicalization: (7686, 21)


[09:44:04] DEPRECATION WARNING: please use MorganGenerator
[09:44:04] DEPRECATION WARNING: please use MorganGenerator
[09:44:04] DEPRECATION WARNING: please use MorganGenerator
[09:44:04] DEPRECATION WARNING: please use MorganGenerator
[09:44:04] DEPRECATION WARNING: please use MorganGenerator
[09:44:04] DEPRECATION WARNING: please use MorganGenerator
[09:44:04] DEPRECATION WARNING: please use MorganGenerator
[09:44:04] DEPRECATION WARNING: please use MorganGenerator
[09:44:04] DEPRECATION WARNING: please use MorganGenerator
[09:44:04] DEPRECATION WARNING: please use MorganGenerator
[09:44:04] DEPRECATION WARNING: please use MorganGenerator
[09:44:04] DEPRECATION WARNING: please use MorganGenerator
[09:44:04] DEPRECATION WARNING: please use MorganGenerator
[09:44:04] DEPRECATION WARNING: please use MorganGenerator
[09:44:04] DEPRECATION WARNING: please use MorganGenerator
[09:44:04] DEPRECATION WARNING: please use MorganGenerator
[09:44:04] DEPRECATION WARNING: please use MorganGenerat

After fingerprints: (7686, 23)
Train/test sizes: (6148, 23) (1538, 23)
Train class counts before SMOTE: {'active': np.int64(1112), 'inactive': np.int64(5036)}
After SMOTE: {'active': 5036, 'inactive': 5036}
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Best params: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': None}
Isotonic calibration succeeded.
ROC-AUC (test): 0.9679056754596324
PR-AUC (test): 0.8917198112199309
Saved rf_fp_model_pro_no_threshold.pkl


[09:52:37] DEPRECATION WARNING: please use MorganGenerator


Test example: NC(=S)c1cccc(-c2cn(C3CCNCC3)c3cc(N)ccc23)c1 {'prob_active': 0.0, 'max_tanimoto': 0.4457831325301205, 'top_matches': [('CN1CC(C(=O)NCc2ccc3c(-c4cccc(C(N)=S)c4)cn(C4CCC(S(=O)(=O)C(F)(F)F)CC4)c3c2)C1', 0.4457831325301205), ('COCc1nc2cnc3cc(-c4c(C)noc4C)c(OC[C@H]4CCNC4)cc3c2n1[C@H](C)c1ccccc1', 0.1650485436893204), ('C=C(C)C(=O)Nc1cccc(C2=NOC3(C2)C[C@@H](C(N)=O)N(C(=O)C2(C)CC2)C3)c1', 0.1595744680851064), ('O=C1N[C@H](c2c(CNCc3ccc4ccn(Cc5ccccc5)c4c3)[nH]c3ccccc23)c2cc(O)ccc21', 0.15151515151515152), ('O=C1NC(=O)c2cccc(NC(=O)C3CC3)c2C1=O', 0.1506849315068493)], 'confidence_advisory': 'medium'}


# NEW MODEL

In [6]:
import pandas as pd

# Load dataset with descriptors
data = pd.read_csv("Descriptors_Specified_SmallGTPases.csv")
print("✅ Dataset loaded:", data.shape)
print(data.head(3))


✅ Dataset loaded: (7686, 21)
  molecule_chembl_id                                   canonical_smiles  \
0       CHEMBL100114  COc1ccc(OCC(=O)N[C@@H](CC(C)C)C(=O)N[C@H]2CC(=...   
1       CHEMBL100405  CC(C)C[C@H](NC(=O)COc1ccc2ccccc2c1)C(=O)N[C@H]...   
2       CHEMBL100409  CC(C)C[C@H](NC(=O)COc1c(Cl)cc(Cl)c2ccccc12)C(=...   

      pIC50 bioactivity_class       MW    LogP  NumHDonors  NumHAcceptors  \
0  5.838632          inactive  444.484  1.5082           3              7   
1  5.000000          inactive  414.458  1.4996           3              6   
2  5.517126          inactive  483.348  2.8064           3              6   

   MolecularWeight  MonoisotopicMW  ...  NumRotatableBonds    TPSA  HBA  HBD  \
0          444.484      444.189651  ...                  9  123.19    7    3   
1          414.458      414.179087  ...                  8  113.96    6    3   
2          483.348      482.101142  ...                  8  113.96    6    3   

   HBA_Lipinski  HBD_Lipinski  RO5_Viola

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Features (drop IDs, SMILES, class column)
X = data.drop(columns=["molecule_chembl_id","canonical_smiles","bioactivity_class"])
y = data["bioactivity_class"].map({"active":1,"inactive":0})  # binary

# Scale features (important for models like LR, SVM; trees don’t strictly need it)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Features shape:", X_scaled.shape, "Labels shape:", y.shape)


Features shape: (7686, 18) Labels shape: (7686,)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

print("Train:", X_train.shape, "Test:", X_test.shape)


Train: (6148, 18) Test: (1538, 18)


In [9]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, classification_report

# Define models
rf = RandomForestClassifier(
    n_estimators=500,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)

xgb = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss",
    scale_pos_weight=(y_train.value_counts()[0]/y_train.value_counts()[1]),
    random_state=42,
    n_jobs=-1
)

# Cross-validation setup
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# RF CV performance
rf_auc = cross_val_score(rf, X_train, y_train, cv=cv, scoring="roc_auc")
print("RF 10-CV ROC-AUC:", rf_auc.mean(), "±", rf_auc.std())

# XGB CV performance
xgb_auc = cross_val_score(xgb, X_train, y_train, cv=cv, scoring="roc_auc")
print("XGB 10-CV ROC-AUC:", xgb_auc.mean(), "±", xgb_auc.std())


RF 10-CV ROC-AUC: 1.0 ± 0.0
XGB 10-CV ROC-AUC: 1.0 ± 4.965068306494546e-17
